In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,26718
2,Huelva,Confirmados PDIA 14 días,1209
3,Huelva,Tasa PDIA 14 días,"235,5944423875129"
4,Huelva,Confirmados PDIA 7 días,630
5,Huelva,Total Confirmados,26906
6,Huelva,Curados,22338
7,Huelva,Fallecidos,357


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  26718.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  6960.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1050 personas en los últimos 7 días 

Un positivo PCR cada 560 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,26718.0,630.0,1209.0,513170.0,122.766335,235.594442,82.0
Condado-Campiña,8038.0,322.0,643.0,156231.0,206.105062,411.570047,52.0
Huelva-Costa,15868.0,256.0,475.0,289548.0,88.413665,164.048793,29.0
Almonte,1206.0,104.0,198.0,24507.0,424.368548,807.932427,17.0
Huelva (capital),6960.0,137.0,257.0,143837.0,95.246703,178.674472,14.0
Cartaya,1128.0,72.0,127.0,20083.0,358.512174,632.375641,12.0
Palos de la Frontera,527.0,23.0,49.0,11742.0,195.878045,417.305399,9.0
Moguer,969.0,44.0,89.0,21867.0,201.216445,407.005991,6.0
Sierra de Huelva-Andévalo Central,2391.0,41.0,69.0,67391.0,60.838984,102.387559,6.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
San Bartolomé de la Torre,268.0,16.0,38.0,3761.0,425.418772,1010.369583,2.0
Almonte,1206.0,104.0,198.0,24507.0,424.368548,807.932427,17.0
Rociana del Condado,432.0,41.0,63.0,7939.0,516.437839,793.550825,5.0
Sanlúcar de Guadiana,6.0,3.0,3.0,403.0,744.416873,744.416873,0.0
Cartaya,1128.0,72.0,127.0,20083.0,358.512174,632.375641,12.0
Bonares,218.0,18.0,35.0,6060.0,297.029703,577.557756,0.0
Lucena del Puerto,181.0,15.0,18.0,3261.0,459.981601,551.977921,0.0
Palos de la Frontera,527.0,23.0,49.0,11742.0,195.878045,417.305399,9.0
Aracena,489.0,21.0,34.0,8255.0,254.391278,411.871593,5.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Trigueros,245.0,4.0,16.0,7862.0,50.877639,203.510557,0.0,0.250000
Aroche,85.0,1.0,4.0,3054.0,32.743942,130.975769,0.0,0.250000
Lepe,2036.0,5.0,16.0,27880.0,17.934003,57.388809,0.0,0.312500
Bollullos Par del Condado,713.0,14.0,44.0,14387.0,97.310072,305.831654,4.0,0.318182
Villalba del Alcor,470.0,3.0,9.0,3366.0,89.126560,267.379679,1.0,0.333333
Villanueva de los Castillejos,156.0,1.0,3.0,2825.0,35.398230,106.194690,0.0,0.333333
Ayamonte,1252.0,2.0,5.0,21104.0,9.476876,23.692191,0.0,0.400000
Gibraleón,556.0,4.0,10.0,12737.0,31.404569,78.511423,1.0,0.400000
San Bartolomé de la Torre,268.0,16.0,38.0,3761.0,425.418772,1010.369583,2.0,0.421053
